### Importing Librairies

In [1]:
# Importing librairies
import pandas as pd
import nltk
import numpy as np
from bs4 import BeautifulSoup
import stop_words
import re, sys
import spacy
from gensim.models import Phrases, phrases
from gensim import models, corpora
import pyLDAvis.gensim
from gensim.models import coherencemodel
from gensim.models.coherencemodel import CoherenceModel
from time import sleep
import sklearn
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
from tqdm import tqdm, tqdm_notebook # progress bars in Jupyter
tqdm.pandas()
tqdm_notebook()

C:\Users\mdegni\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

0it [00:00, ?it/s]

In [2]:
data = pd.read_csv('request_tag.csv')

In [3]:
data.head(2)

,Unnamed: 0,Body,Title,Tags,token_tag,Questions,token_questions,tokens_clean,tokens_clean_lemma
0,0,"<p>I want to use a track-bar to change a form's opacity.</p>\r\n\r\n<p>This is my code:</p>\r\n\r\n<pre><code>decimal trans = trackBar1.Value / 5000;\r\nthis.Opacity = trans;\r\n</code></pre>\r\n\r\n<p>When I build the application, it gives the following error:</p>\r\n\r\n<blockquote>\r\n <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p>\r\n</blockquote>\r\n\r\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\r\n",Convert Decimal to Double?,<c#><floating-point><type-conversion><double><decimal>,"['c#', 'floating-point', 'type-conversion', 'double', 'decimal']","Convert Decimal to Double? <p>I want to use a track-bar to change a form's opacity.</p>\r\n\r\n<p>This is my code:</p>\r\n\r\n<pre><code>decimal trans = trackBar1.Value / 5000;\r\nthis.Opacity = trans;\r\n</code></pre>\r\n\r\n<p>When I build the application, it gives the following error:</p>\r\n\r\n<blockquote>\r\n <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p>\r\n</blockquote>\r\n\r\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\r\n","['convert', 'decimal', 'to', 'double', 'i', 'want', 'to', 'use', 'a', 'track', 'bar', 'to', 'change', 'a', 'form', 's', 'opacity', 'this', 'is', 'my', 'code', 'when', 'i', 'build', 'the', 'application', 'it', 'gives', 'the', 'following', 'error', 'cannot', 'implicitly', 'convert', 'type', 'to', 'i', 'tried', 'using', 'and', 'but', 'then', 'the', 'control', 'doesn', 't', 'work', 'this', 'code', 'worked', 'fine', 'in', 'a', 'past', 'vb', 'net', 'project']","['convert', 'decimal', 'double', 'want', 'use', 'track', 'bar', 'change', 'form', 'opacity', 'code', 'build', 'application', 'gives', 'following', 'error', 'implicitly', 'convert', 'type', 'tried', 'using', 'control', 'doesn', 'work', 'code', 'worked', 'fine', 'past', 'vb', 'net', 'project']","['convert', 'decimal', 'double', 'want', 'use', 'track', 'bar', 'change', 'form', 'opacity', 'code', 'build', 'application', 'give', 'follow', 'error', 'implicitly', 'convert', 'type', 'try', 'use', 'control', 'doesn', 'work', 'code', 'work', 'fine', 'past', 'vb', 'net', 'project']"
1,1,"<p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to '0' width on <a href=""http://en.wikipedia.org/wiki/Internet_Explorer_7"" rel=""noreferrer"">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p>\r\n\r\n<p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width on the child works.</p>\r\n\r\n<ol>\r\n<li>Is there something I'm missing here?</li>\r\n<li>Is there an easy fix for this besides the <em>pixel-based width</em> on the\r\nchild?</li>\r\n<li>Is there an area of the CSS specification that covers this?</li>\r\n</ol>\r\n",Percentage width child element in absolutely positioned parent on Internet Explorer 7,<html><css><css3><internet-explorer-7>,"['html', 'css', 'css3', 'internet-explorer-7']","Percentage width child element in absolutely positioned parent on Internet Explorer 7 <p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to '0' width on <a href=""http://en.wikipedia.org/wiki/Internet_Explorer_7"" rel=""noreferrer"">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p>\r\n\r\n<p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width

In [4]:
def pass_list(df, col):

    for x in col:
        l=[]
        for j in range(len(df)):
            l.append(str(df[x].loc[j]).replace('[',"").replace(']',"").replace("'","").split(', '))
        df[x]=l
#     return df[x]
# data['tokens_clean_lemma'].loc[0].replace('[',"").replace(']',"").replace("'","").split(', ')

In [5]:
pass_list(data, ['token_tag','token_questions','tokens_clean','tokens_clean_lemma'])

In [6]:
data.head(2)

,Unnamed: 0,Body,Title,Tags,token_tag,Questions,token_questions,tokens_clean,tokens_clean_lemma
0,0,"<p>I want to use a track-bar to change a form's opacity.</p>\r\n\r\n<p>This is my code:</p>\r\n\r\n<pre><code>decimal trans = trackBar1.Value / 5000;\r\nthis.Opacity = trans;\r\n</code></pre>\r\n\r\n<p>When I build the application, it gives the following error:</p>\r\n\r\n<blockquote>\r\n <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p>\r\n</blockquote>\r\n\r\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\r\n",Convert Decimal to Double?,<c#><floating-point><type-conversion><double><decimal>,"[c#, floating-point, type-conversion, double, decimal]","Convert Decimal to Double? <p>I want to use a track-bar to change a form's opacity.</p>\r\n\r\n<p>This is my code:</p>\r\n\r\n<pre><code>decimal trans = trackBar1.Value / 5000;\r\nthis.Opacity = trans;\r\n</code></pre>\r\n\r\n<p>When I build the application, it gives the following error:</p>\r\n\r\n<blockquote>\r\n <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p>\r\n</blockquote>\r\n\r\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\r\n","[convert, decimal, to, double, i, want, to, use, a, track, bar, to, change, a, form, s, opacity, this, is, my, code, when, i, build, the, application, it, gives, the, following, error, cannot, implicitly, convert, type, to, i, tried, using, and, but, then, the, control, doesn, t, work, this, code, worked, fine, in, a, past, vb, net, project]","[convert, decimal, double, want, use, track, bar, change, form, opacity, code, build, application, gives, following, error, implicitly, convert, type, tried, using, control, doesn, work, code, worked, fine, past, vb, net, project]","[convert, decimal, double, want, use, track, bar, change, form, opacity, code, build, application, give, follow, error, implicitly, convert, type, try, use, control, doesn, work, code, work, fine, past, vb, net, project]"
1,1,"<p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to '0' width on <a href=""http://en.wikipedia.org/wiki/Internet_Explorer_7"" rel=""noreferrer"">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p>\r\n\r\n<p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width on the child works.</p>\r\n\r\n<ol>\r\n<li>Is there something I'm missing here?</li>\r\n<li>Is there an easy fix for this besides the <em>pixel-based width</em> on the\r\nchild?</li>\r\n<li>Is there an area of the CSS specification that covers this?</li>\r\n</ol>\r\n",Percentage width child element in absolutely positioned parent on Internet Explorer 7,<html><css><css3><internet-explorer-7>,"[html, css, css3, internet-explorer-7]","Percentage width child element in absolutely positioned parent on Internet Explorer 7 <p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to '0' width on <a href=""http://en.wikipedia.org/wiki/Internet_Explorer_7"" rel=""noreferrer"">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p>\r\n\r\n<p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width on the child works.</p>\r\n\r\n<ol>\r\n<li>Is there something I'm missing here?</li>\r\n<li>Is there an easy fix for this besides the <em>pixel-based width</em> on the\r\nchild?</li>\r\n<li>Is there an area of the CSS specification that covers this?</li>\

### Build corpus with Back of Word

In [7]:
# Define stopwords
sw = stop_words.get_stop_words(language='en')

In [8]:
# Use bigram and trigram to catch combination of 2/3 words that have a specific meaning together
tokens_lemma = data['tokens_clean_lemma'].tolist()
bigram_lemma =  Phrases(tokens_lemma, min_count = 5)
trigram_lemma = Phrases(bigram_lemma[tokens_lemma])
# for faster implementation
bigram_mod_lemma = phrases.Phraser(bigram_lemma)
trigram_mod_lemma = phrases.Phraser(trigram_lemma)

tokens_lemma = list(trigram_mod_lemma[bigram_mod_lemma[tokens_lemma]])
tokens_lemma = [[token.lower() for token in t if token.lower() not in sw] for t in tokens_lemma]
tokens_lemma

[['convert',
  'decimal',
  'double',
  'want',
  'use',
  'track',
  'bar',
  'change',
  'form',
  'opacity',
  'code',
  'build',
  'application',
  'give',
  'follow_error',
  'implicitly',
  'convert',
  'type',
  'try',
  'use',
  'control',
  'doesn_work',
  'code',
  'work_fine',
  'past',
  'vb_net',
  'project'],
 ['percentage',
  'width',
  'child_element',
  'absolutely_position',
  'parent',
  'internet_explorer',
  'absolutely_position',
  'contain',
  'several',
  'child',
  'one',
  'relatively',
  'position',
  'use',
  'percentage',
  'base',
  'width',
  'child',
  'collapse',
  'width',
  'internet_explorer',
  'firefox_safari',
  'use',
  'pixel',
  'width',
  'work',
  'parent',
  'relatively',
  'position',
  'percentage',
  'width',
  'child',
  'work',
  'something_miss',
  'easy',
  'fix',
  'besides',
  'pixel',
  'base',
  'width',
  'child',
  'area',
  'css',
  'specification',
  'cover'],
 ['calculate',
  'someone',
  'age',
  'give',
  'represent',
  'pe

### Training unsupervised model (LDA)

In [9]:
# dictionary
dictionary_LDA_lemma = corpora.Dictionary(tokens_lemma)
dictionary_LDA_lemma.filter_extremes(no_below=5, no_above=0.9)
corpus_lemma = [dictionary_LDA_lemma.doc2bow(tok) for tok in tokens_lemma]
print(dictionary_LDA_lemma)

Dictionary(4325 unique tokens: ['application', 'bar', 'build', 'change', 'code']...)


In [10]:
num_topics = 30
%time lda_model_lemma = models.LdaModel(corpus_lemma, num_topics=num_topics, \
                                  id2word=dictionary_LDA_lemma, \
                                  passes=10, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA_lemma.keys()))

Wall time: 58.7 s


In [11]:
for i,topic in lda_model_lemma.show_topics(formatted=True, num_topics=3, num_words=5):
    print(str(i)+": "+ topic)
    print()

21: 0.043*"javascript" + 0.039*"page" + 0.030*"html" + 0.017*"use" + 0.017*"work"

25: 0.032*"control" + 0.031*"application" + 0.027*"set" + 0.024*"bit" + 0.021*"form"

8: 0.039*"change" + 0.025*"svn" + 0.023*"file" + 0.022*"update" + 0.019*"subversion"



### Visualization

In [12]:
pyLDAvis.enable_notebook()
vis_lemma = pyLDAvis.gensim.prepare(lda_model_lemma, corpus_lemma, dictionary_LDA_lemma)
vis_lemma

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
17     6.744794  1        1      -0.069345 -0.029006
3      6.666475  1        2      -0.093187  0.121654
14     6.002230  1        3      -0.135672 -0.029514
2      5.796878  1        4       0.057307  0.110364
9      4.984464  1        5      -0.065256 -0.101832
27     3.911660  1        6      -0.062335  0.063831
19     3.905817  1        7       0.112708  0.032058
21     3.782131  1        8       0.100526  0.002563
7      3.619905  1        9       0.115695  0.090181
13     3.279832  1        10      0.072954 -0.122938
11     3.177326  1        11     -0.083232 -0.033985
23     3.124917  1        12      0.081508  0.079084
24     3.113204  1        13     -0.158918  0.156345
18     3.099281  1        14      0.007007 -0.006362
4      3.022288  1        15     -0.165147  0.006222
6      2.944196  1        16     -0.044704 -0.046794
5      2.793316  1        17      0.033475 -0.086735
10     2.748970  1        18      0.127068 -0.038031
28     2.747971  1        19      0.148388 -0.008402
22     2.590392  1        20     -0.063270  0.155084
12     2.577946  1        21      0.064892 -0.113228
0      2.472286  1        22      0.117068 -0.001299
8      2.436704  1        23      0.024159 -0.150260
1      2.328351  1        24     -0.106700 -0.056251
20     2.327654  1        25     -0.202683 -0.052598
29     2.266949  1        26      0.068237 -0.091930
15     2.182911  1        27     -0.004053  0.010720
16     2.049031  1        28      0.013193  0.161503
25     1.724023  1        29     -0.046868 -0.128436
26     1.578095  1        30      0.157182  0.107993, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
183   Default  1801.000000  file           1801.000000  30.0000  30.0000
230   Default  996.000000   database       996.000000   29.0000  29.0000
15    Default  1024.000000  project        1024.000000  28.0000  28.0000
145   Default  591.000000   test           591.000000   27.0000  27.0000
156   Default  711.000000   table          711.000000   26.0000  26.0000
2     Default  686.000000   build          686.000000   25.0000  25.0000
281   Default  817.000000   class          817.000000   24.0000  24.0000
5     Default  662.000000   control        662.000000   23.0000  23.0000
203   Default  752.000000   object         752.000000   22.0000  22.0000
285   Default  580.000000   library        580.000000   21.0000  21.0000
323   Default  860.000000   page           860.000000   20.0000  20.0000
495   Default  854.000000   window         854.000000   19.0000  19.0000
954   Default  423.000000   python         423.000000   18.0000  18.0000
11    Default  501.000000   form           501.000000   17.0000  17.0000
71    Default  1031.000000  user           1031.000000  16.0000  16.0000
0     Default  1355.000000  application    1355.000000  15.0000  15.0000
69    Default  428.000000   string         428.000000   14.0000  14.0000
668   Default  441.000000   javascript     441.000000   13.0000  13.0000
389   Default  825.000000   method         825.000000   12.0000  12.0000
1493  Default  373.000000   image          373.000000   11.0000  11.0000
583   Default  348.000000   property       348.000000   10.0000  10.0000
494   Default  439.000000   visual_studio  439.000000   9.0000   9.0000 
231   Default  890.000000   datum          890.000000   8.0000   8.0000 
257   Default  1205.000000  run            1205.000000  7.0000   7.0000 
263   Default  355.000000   sql_server     355.000000   6.0000   6.0000 
3     Default  877.000000   change         877.000000   5.0000   5.0000 
62    Default  593.000000   value          593.000000   4.0000   4.0000 
236   Default  659.000000   function       659.000000   3.0000   3.0000 
55    Default  393.000000   display        393.000000   2.0000   2.0000

### Model performance: LDA using Lemmatization

In [13]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_lemma.log_perplexity(corpus_lemma))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda_lemma = CoherenceModel(model=lda_model_lemma, texts=tokens_lemma, dictionary=dictionary_LDA_lemma, coherence='c_v')
coherence_lda_lemma = coherence_model_lda_lemma.get_coherence()
print('\nCoherence Score: ', coherence_lda_lemma)


Perplexity:  -7.83289219324

Coherence Score:  0.325821400642


### Fine-tuning the model

#### Finding optimal number of topics

In [14]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    perplexity_values = []
    coherence_values = []
    model_list = []
    with tqdm(total=len(range(start, limit, step)), file=sys.stdout) as pbar:
        for num_topics in range(start, limit, step):
            #model = models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary)
            model = models.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=10, alpha=0.001, eta=0.001, random_state = 42)
            model_list.append(model)
            perplexity_values.append(model.log_perplexity(corpus))
            model_coherence = coherencemodel.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
            coherence_values.append(model_coherence.get_coherence())
            pbar.update(1)
            sleep(1)

    return perplexity_values, model_list, coherence_values

In [15]:
# Can take a long time to run.
k_min = 2
k_max = 50
pas = 2
perplexity_values, model_list, coherence_values = compute_coherence_values(dictionary=dictionary_LDA_lemma, corpus=corpus_lemma,
                                                                           texts=tokens_lemma, start=k_min, limit=k_max, step=pas)

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [29:53<00:00, 80.90s/it]


In [16]:
min(perplexity_values)

-11.720454242721123

In [17]:
coherence_values

[0.33590592711721678,
 0.35561872449176674,
 0.34455468314659049,
 0.38458996176068649,
 0.35359024890915158,
 0.34869733937323533,
 0.35706795509041472,
 0.34143280002298493,
 0.32673710870379191,
 0.33703064886914957,
 0.33271997749444954,
 0.34197107752168998,
 0.31852440979856422,
 0.31366666761095702,
 0.31721053249781128,
 0.3192335582919551,
 0.3192917285384918,
 0.31662769634712862,
 0.32034237081790268,
 0.30890622386604605,
 0.3206146421480382,
 0.30670449016596135,
 0.31194156711777632,
 0.30451556007975716]

In [18]:
## Best model
num_topics = 50
%time lda_model_lemma_new = models.LdaModel(corpus_lemma, num_topics=num_topics, \
                                  id2word=dictionary_LDA_lemma, \
                                  passes=10, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA_lemma.keys()))

Wall time: 1min 3s


#### Finding the dominant topic in each sentence

In [19]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [20]:
df_topic_sents_tag= format_topics_sentences(ldamodel=lda_model_lemma_new, corpus=corpus_lemma, texts=tokens_lemma)

# Format
df_dominant_topic = df_topic_sents_tag.reset_index()
df_dominant_topic.columns = ['Comment_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Comment_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4.0,0.5649,"code, error, get, use, thread, line, try, follow, comment, result","[convert, decimal, double, want, use, track, bar, change, form, opacity, code, build, application, give, follow_error, implicitly, convert, type, try, use, control, doesn_work, code, work_fine, past, vb_net, project]"
1,1,1.0,0.8294,"call, merge, update, change, entity, problem, child, job, parent, get","[percentage, width, child_element, absolutely_position, parent, internet_explorer, absolutely_position, contain, several, child, one, relatively, position, use, percentage, base, width, child, collapse, width, internet_explorer, firefox_safari, use, pixel, width, work, parent, relatively, position, percentage, width, child, work, something_miss, easy, fix, besides, pixel, base, width, child, area, css, specification, cover]"
2,2,2.0,0.5624,"list, string, number, str, can, example, way, like, element, integer","[calculate, someone, age, give, represent, person, birthday, calculate, age, year]"
3,3,27.0,0.4046,"server, db, day, need, new, monitor, rail, net, ajax, think","[calculate, relative, time, give, specific, value, display, relative, time, like, hour, ago, day, ago, month_ago]"
4,4,20.0,0.3904,"namespace, video, product, category, user, expose, shell, foreign_key, can, relationship","[determine, user, timezone, standard, way, web_server, able, determine, user, timezone, within, web_page, perhaps, http_header, part, user_agent, string]"
5,5,7.0,0.3092,"python, cache, use, can, view, encrypt, beta, empty, jsp, will","[difference, math, floor, math, truncate, difference, net]"
6,6,20.0,0.7260,"namespace, video, product, category, user, expose, shell, foreign_key, can, relationship","[fill, dataset, datatable, linq_query, result_set, expose, linq_query, asmx, web_service, usually, business, tier, can, return, type, can, serialize, transport, asmx, can, linq_query, way, populate, type, via, linq_query, can, get, result_set, linq_query, alternatively, linq_query, serializeable, can, expose, asmx, web_service]"
7,7,9.0,0.5504,"table, database, query, sql, datum, sql_server, field, column, mysql, use","[binary, data, mysql, store, binary, data, mysql]"
8,8,4.0,0.6029,"code, error, get, use, thread, line, try, follow, comment, result","[fast_way, get, value, look, fast_way, obtain, value, personal, challenge, specifically, use, way, don, involve, use, constant, like, hard_cod, number, program, test, various, way, know, inline, assembly_version, theory, fast, option, though, clearly, portable, ve, include, baseline, compare, version, test, build, version, fast, gcc, auto, fold, constant, specify, version, fast, main, testing, program, inline, assembly, stuff, will, work, system, build_script, build, configuration, test, apart, test, various, compiler, flag, ve, compare, bit, bit, optimisation, different, ve, also, try, switching, order, test, around, still, version, still, come, top, every_time]"
9,9,4.0,0.4542,"code, error, get, use, thread, line, try, follow, comment, result","[throw, error, mysql, trigger, table, can, throw, error, prevent, update, table]"


In [21]:
df_dominant_topic.tail(10)

,Comment_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
5512,5512,12.0,0.4467,"array, difference, index, action, code, mapping, label, use, one, loop","[difference, union, union, difference]"
5513,5513,23.0,0.2488,"java, column, performance, add, number, limit, row, can, time, many","[open_source, alternative, matlab, fmincon, function, open_source, alternative, matlab, function, constrain, linear, optimization, rewrite, matlab, program, use, python, numpy, scipy, function, haven_find, equivalent, numpy, base, solution, ideal, language, will]"
5514,5514,16.0,0.2592,"multiple, sharepoint, can, server, trigger, one, account, insert, keyword, error_message","[come, bit, kernel, can, run, bit, binary, os, box, kernel, bit, binary, yet, can, run, bit, binary, work, cristi, diciu, file, mach, bit, executable, cristi, diciu, file, mach, kernel, mach, kernel, mach, universal, binary, architecture, mach, kernel, architecture, mach, executable, mach, kernel, architecture, ppc, mach, executable, ppc, cristi, diciu, cristi, diciu, echo]"
5515,5515,42.0,0.2224,"use, net, client, application, look, component, dynamic, service, web_service, good","[dynamic, code_analysis, dynamic, code_analysis, different, static_code_analysis, ie, can, catch, can, catch, static, ve_hear, bound, check, memory, analysis, thing, check, use, dynamic, analysis, adam]"
5516,5516,47.0,0.6140,"object, method, call, get, use, collection, way, vb, create, request","[garbage, collection, necessary, set, large, object, null, dispose, method, necessary, set, large, object, implement, method]"
5517,5517,39.0,0.3291,"java, use, can, method, programming, good, know, answer, code, think","[task, scheduler, interesting, discussion, colleague, good, scheduling, strategy, realtime, task, everyone, good, understanding, common, useful, scheduling, strategy, answer, please, choose, one, strategy, go, detail, rather, give, little, info, several, strategy, something, add, someone_else, description, short, add, comment, rather, new, answer, long, useful, simply, much, good, description, please, use, answer, strategy, describe, general, case, assume, people, know, task, queue, semaphore, lock, os, fundamental, outside, scheduler, strategy, optimize, task, latency, efficiency, realtime, jitter, resource, sharing, etc, realtime, can, make, realtime, current, strategy, priority, base, preemptive, low, power, slow, clock, adam]"
5518,5518,29.0,0.4744,"search, text, box, edit, use, cursor, div, width, podcast, height","[count, list, cell, background_color, cell, contain, text, background_color, cell, blue, red, function, use, count_number, red, cell, try, success, red]"
5519,5519,13.0,0.2298,"window, wpf, device, get, drive, vista, use, can, disable, system","[remove, black, border, around, hyperlink, image, turn, image, hyperlink, wrap, firefox, add, black, border, around, image, safari, display, border, css, declaration, best, eliminate, border]"
5520,5520,18.0,0.4334,"download, path, install, find, import, mac, file, can, unix, look","[order, location, search, load, reference, dll, know, net_framework, look, reference, dll, several, location, global_assembly_cache, gac, private, path, add, appdomain, current, directory, execute, assembly, order, location, search, search, dll, cease, match, find, continue, location, conflict, resolve, also, please, confirm, deny, location, provide, location, fail, mention]"
5521,5521,24.0,0.3480,"command, http, can, call, request, use, get, response, program, work","[really, kill_process, window, occasionally, program, window, machine, go, crazy, just, hang, will, call, task_manager, hit, end, process, button, however, doesn, always, work, try, enough, time, will, usually, die, eventually, really, like_able, just, kill, immediately, linux, just, guarantee, process, will, die, also, use, writing, batch, script, write, batch, script, programming, program, command, come, window, will, always, kill_process, free, third_party, app, fine, although, prefer

In [22]:
### Labelling topics with tags
data_sup_2 = pd.concat([df_dominant_topic['Dominant_Topic'], data['token_tag']], axis=1)
data_sup_2.head()

,Dominant_Topic,token_tag
0,4.0,"[c#, floating-point, type-conversion, double, decimal]"
1,1.0,"[html, css, css3, internet-explorer-7]"
2,2.0,"[c#, .net, datetime]"
3,27.0,"[c#, datetime, time, datediff, relative-time-span]"
4,20.0,"[javascript, html, browser, timezone, timezoneoffset]"


In [23]:
# Topic Encoding 
data_sup_2 = pd.concat([pd.get_dummies(data_sup_2['Dominant_Topic'], prefix='Topic', drop_first=True), data_sup_2['token_tag']], axis=1)
data_sup_2.columns

Index(['Topic_1.0', 'Topic_2.0', 'Topic_3.0', 'Topic_4.0', 'Topic_5.0',
       'Topic_6.0', 'Topic_7.0', 'Topic_8.0', 'Topic_9.0', 'Topic_10.0',
       'Topic_11.0', 'Topic_12.0', 'Topic_13.0', 'Topic_14.0', 'Topic_15.0',
       'Topic_16.0', 'Topic_17.0', 'Topic_18.0', 'Topic_19.0', 'Topic_20.0',
       'Topic_21.0', 'Topic_22.0', 'Topic_23.0', 'Topic_24.0', 'Topic_25.0',
       'Topic_26.0', 'Topic_27.0', 'Topic_28.0', 'Topic_29.0', 'Topic_30.0',
       'Topic_31.0', 'Topic_32.0', 'Topic_33.0', 'Topic_34.0', 'Topic_35.0',
       'Topic_36.0', 'Topic_37.0', 'Topic_38.0', 'Topic_39.0', 'Topic_40.0',
       'Topic_41.0', 'Topic_42.0', 'Topic_43.0', 'Topic_44.0', 'Topic_45.0',
       'Topic_46.0', 'Topic_47.0', 'Topic_48.0', 'Topic_49.0', 'token_tag'],
      dtype='object')

In [26]:
data_sup_2

,Topic_1.0,Topic_2.0,Topic_3.0,Topic_4.0,Topic_5.0,Topic_6.0,Topic_7.0,Topic_8.0,Topic_9.0,Topic_10.0,...,Topic_41.0,Topic_42.0,Topic_43.0,Topic_44.0,Topic_45.0,Topic_46.0,Topic_47.0,Topic_48.0,Topic_49.0,token_tag
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[c#, floating-point, type-conversion, double, decimal]"
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[html, css, css3, internet-explorer-7]"
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[c#, .net, datetime]"
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[c#, datetime, time, datediff, relative-time-span]"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[javascript, html, browser, timezone, timezoneoffset]"
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,"[.net, math]"
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[c#, linq, web-services, .net-3.5]"
7,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,"[mysql, database, binary-data, data-storage]"
8,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[performance, algorithm, language-agnostic, unix, pi]"
9,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[mysql, database, triggers]"


In [27]:
# Clean token_tag
def trans(val):
    val = " ".join(str(x) for x in val)
    return val
data_sup_2['token_tag'] = data_sup_2['token_tag'].apply(trans)

In [28]:
data_sup_2

,Topic_1.0,Topic_2.0,Topic_3.0,Topic_4.0,Topic_5.0,Topic_6.0,Topic_7.0,Topic_8.0,Topic_9.0,Topic_10.0,...,Topic_41.0,Topic_42.0,Topic_43.0,Topic_44.0,Topic_45.0,Topic_46.0,Topic_47.0,Topic_48.0,Topic_49.0,token_tag
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c# floating-point type-conversion double decimal
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,html css css3 internet-explorer-7
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c# .net datetime
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c# datetime time datediff relative-time-span
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,javascript html browser timezone timezoneoffset
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,.net math
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c# linq web-services .net-3.5
7,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,mysql database binary-data data-storage
8,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,performance algorithm language-agnostic unix pi
9,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mysql database triggers


In [30]:
# ## Question/ Tags
# len(data['token_tag'])
# def trans(val):
#     val = " ".join(str(x) for x in val)
#     return val
# question = data['token_tag'].apply(trans)

# from sklearn.feature_extraction.text import TfidfVectorizer
# vec = TfidfVectorizer()
# questions_tag_matr = vec.fit_transform(question).todense()

# # matrix tf-idf
# questions_tag_matr = pd.DataFrame(data=questions_tag_matr,columns=vec.get_feature_names())
# tf_idfmatrix.head()

In [31]:
# questions_tag_matr.shape

In [32]:
# unique_words = list(unique(" ".join(question).split(" ")))

In [33]:
# len(unique_words)

In [34]:
# question[1]

### Supervised model

In [47]:
# Splitting data in training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_sup_2.iloc[:,:-1].values, data_sup_2['token_tag'], test_size=0.3)

In [48]:
# List of tags
# 1. Define a list of token values from dataframe
list_tok = data.token_tag.values
list_tok

# 2. Put all element in one list
list_tag = []
for sublist in list_tok:
    for item in sublist:
        list_tag.append(item)

all_tags = sorted(set(" ".join(t for t in list_tag).split(' ')))

if '' in all_tags: all_tags.remove('')
if ' ' in all_tags: all_tags.remove(' ')

In [49]:
# Binarizing target 
from sklearn import preprocessing
mlb = preprocessing.MultiLabelBinarizer(classes=all_tags)
mlb.fit([w.split() for w in y_train])

y_train_mlb = mlb.transform([w.split() for w in y_train])
y_test_mlb = mlb.transform([w.split() for w in y_test])

#### Multinomial Naives Bayes

In [179]:
# Training multinomial Naives Bayes model
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier

m_NB = OneVsRestClassifier(MultinomialNB()).fit(X_train, y_train_mlb)
y_pred_NB = m_NB.predict_proba(X_test)
# y_pred_class = m_NB.predict(X_test)

In [183]:
ffgd = pd.DataFrame(data=y_pred_prob[0], index=all_tags)#.sort(ascending=False)
ffgd.columns = ['Probabilities']
ffgd.sort_values(by='Probabilities', ascending=False).head()

,Probabilities
c#,0.196276
.net,0.120634
c++,0.111925
php,0.074453
c,0.070478


In [181]:
# logistic regression
from sklearn.linear_model import LogisticRegression

LR = OneVsRestClassifier(LogisticRegression()).fit(X_train, y_train_mlb)
y_pred_prob = LR.predict_proba(X_test)

lr_df = pd.DataFrame(data=y_pred_prob[0], index=all_tags)#.sort(ascending=False)
lr_df.columns = ['Probability']
lr_df.sort_values(by='Probability', ascending=False).head()

,Probability
c#,0.196276
.net,0.120634
c++,0.111925
php,0.074453
c,0.070478


### Model Performance -- Metrics

In [117]:
# define the percentage of true value inside observed values

def metrics():
    L=[]
    xf = len(y_test[0])
    for i in lr_df.sort_values(by='Probability', ascending=False).index[:xf]:
        if i in y_test[0]:
            L.append(i)
    A = len(L)
    score = A/xf

In [187]:
def metrics(y_predict_prob, y, classes):
    final_score=[]
    for i, j in zip(range(len(y)+1), y.index):
        df = pd.DataFrame(data=y_predict_prob[i], index=all_tags)
        df.columns = ['Probabilities']
        a= df.sort_values(by='Probabilities', ascending=False).index[:len(y[j])]
        b = y[j].split() 
        L = [i for i, j in zip(a, b) if i == j]
        score = len(L)/len(b)
        final_score.append(score)
    return np.mean(final_score)

In [188]:
metrics(y_pred_prob, y_test, all_tags)

0.065007040836853755

In [189]:
metrics(y_pred_NB, y_test, all_tags)

0.061677730838865415